In [2]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import pandas as pd
import psycopg2 as pg

class spotWrapper():
    def __init__(self):
        client_id = 'accd0aa479164ddcbf1cbf822512b80b'
        client_secret = '58bfc467435045e7b61c86fb03385729'
        client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
        self.spot = spotipy.Spotify(client_credentials_manager=client_credentials_manager)
    def getAlbums(self, uri):
        return self.spot.artist_albums(uri, album_type="album")
    
def connect():
    connection_var = pg.connect(
        host="localhost",
        database="hiphop",
        user="garcgabe",
        password="password"
    )
    return connection_var




In [3]:
spot = spotWrapper()
spot_artists = pd.read_excel("/Users/garcgabe/Desktop/SpotifyArtists.xlsx").drop("Unnamed: 0", axis=1)
artists_uri = spot_artists[['spotify_name','artist_uri']]


In [4]:
# title of each column
columns = ["album_uri", "album_name", "total_tracks", "release_date", 
           "artist_uris", "artist_names"]

# init empty lists to load then concat into DF
album_uri, album_name, total_tracks, release_date, artist_uris, artist_names  = ([] for i in range (6))
for i in range(0,len(artists_uri)-25):
    name_uri = artists_uri["artist_uri"][i]
    search_tree = spot.getAlbums(name_uri)['items']
    for j in range(0,len(search_tree)):
        search_tree_split = search_tree[j]
        all_artists, all_uris = ([] for x in range(2))
        number_of_artists = len(search_tree_split["artists"])
        for artist_num in range(0,number_of_artists):
            artist_name_in_list = search_tree_split["artists"][artist_num]["name"]
            try:
                artist_uri_in_list = spot_artists.loc[spot_artists["spotify_name"] == artist_name_in_list, "artist_uri"].iloc[0]
            except:
                artist_uri_in_list = "0"
            all_uris.append(artist_uri_in_list)
            all_artists.append(artist_name_in_list)
        artist_names.append(", ".join(all_artists))
        artist_uris.append(", ".join(all_uris))
        album_uri.append(search_tree_split["uri"])
        album_name.append(search_tree_split["name"].strip().split("\n")[0])
        total_tracks.append(str(search_tree_split["total_tracks"]))
        release_date.append(search_tree_split["release_date"])
final_df = pd.concat([pd.Series(album_uri), pd.Series(album_name), pd.Series(total_tracks),
                                    pd.Series(release_date), pd.Series(artist_uris), pd.Series(artist_names)], 
                                    axis=1, keys=columns)
        
    
        



In [6]:
# title of each column
columns = ["album_uri", "album_name", "total_tracks", "release_date", 
           "artist_uris", "artist_names"]

connection = connect()
curr = connection.cursor()
curr.execute("""
    CREATE TABLE IF NOT EXISTS D_Albums (
        album_uri varchar(255) primary key,
        album_name varchar(255),
        total_tracks integer, 
        release_date varchar(255),
        artist_uris text,
        artist_names text
    )
""")
connection.commit()
for row_num in range(0,len(final_df)):
    curr.execute("""
        insert into D_Albums(album_uri, album_name, total_tracks, release_date, artist_uris, artist_names)
        values(%s, %s, %s, %s, %s, %s)
        """,
        final_df.iloc[row_num])
    print(row_num)
    connection.commit()
curr.close()
connection.close()

UniqueViolation: duplicate key value violates unique constraint "d_albums_pkey"
DETAIL:  Key (album_uri)=(spotify:album:3MATDdrpHmQCmuOcozZjDa) already exists.


In [19]:
counts = final_df.value_counts("album_uri")
final_df[final_df["album_uri"].isin(counts[counts>1].index)].sort_values("album_uri")

,album_uri,album_name,total_tracks,release_date,artist_uris,artist_names
44,spotify:album:0C0Vs4XobImmqpr6kIasde,SUPERFLY (Original Motion Picture Soundtrack),23,2018-06-15,"spotify:artist:1RyvyyTE3xzB2ZywiAwp0i, spotify...","Future, Lil Wayne"
114,spotify:album:0C0Vs4XobImmqpr6kIasde,SUPERFLY (Original Motion Picture Soundtrack),23,2018-06-15,"spotify:artist:1RyvyyTE3xzB2ZywiAwp0i, spotify...","Future, Lil Wayne"
293,spotify:album:0OcMap99vLEeGkBCfCwRwS,Watch The Throne,12,2011-08-08,"spotify:artist:3nFkdlSjzX9mRTtwJOzDYB, spotify...","JAY-Z, Kanye West"
274,spotify:album:0OcMap99vLEeGkBCfCwRwS,Watch The Throne,12,2011-08-08,"spotify:artist:3nFkdlSjzX9mRTtwJOzDYB, spotify...","JAY-Z, Kanye West"
295,spotify:album:0eZRIt3Ht1IuMXsr0QR8OY,Watch The Throne,12,2011-08-08,"spotify:artist:3nFkdlSjzX9mRTtwJOzDYB, spotify...","JAY-Z, Kanye West"
276,spotify:album:0eZRIt3Ht1IuMXsr0QR8OY,Watch The Throne,12,2011-08-08,"spotify:artist:3nFkdlSjzX9mRTtwJOzDYB, spotify...","JAY-Z, Kanye West"
270,spotify:album:1YwzJz7CrV9fd9Qeb6oo1d,Watch The Throne (Deluxe),16,2011-08-08,"spotify:artist:3nFkdlSjzX9mRTtwJOzDYB, spotify...","JAY-Z, Kanye West"
289,spotify:album:1YwzJz7CrV9fd9Qeb6oo1d,Watch The Throne (Deluxe),16,2011-08-08,"spotify:artist:3nFkdlSjzX9mRTtwJOzDYB, spotify...","JAY-Z, Kanye West"
41,spotify:album:1uCLzanq1xy3eX2zyM4Sr0,Future & Juice WRLD Present... WRLD ON DRUGS,16,2018-10-19,"spotify:artist:1RyvyyTE3xzB2ZywiAwp0i, spotify...","Future, Juice WRLD"
156,spotify:album:1uCLzanq1xy3eX2zyM4Sr0,Future & Juice WRLD Present... WRLD ON DRUGS,16,2018-10-19,"spotify:artist:1RyvyyTE3xzB2ZywiAwp0i, spotify...","Future, Juice WRLD"


In [16]:
counts[counts>1].index

Index(['spotify:album:4P63UgNDUcF11MnWzyvVrh',
       'spotify:album:4z8IoUDFp5dmiQNDlT4hu5',
       'spotify:album:3ihwKkIMJWmmp1huNH0iWC',
       'spotify:album:2yXnY2NiaZk9QiJJittS81',
       'spotify:album:0eZRIt3Ht1IuMXsr0QR8OY',
       'spotify:album:3aITAVBURujVe8fhI2seeR',
       'spotify:album:3XADnbi4uhYXb7RuSJ7bre',
       'spotify:album:27fzM2E0lgovCD7PCq6eh4',
       'spotify:album:2AB0tGyHVCLsZoGGkzufTt',
       'spotify:album:6P9PZjWXoCRF5b66BafPKY',
       'spotify:album:6Oa0ZYzr4VdFIz3zNmYJYQ',
       'spotify:album:41NvzpIHKpaYHKnh1bF3TN',
       'spotify:album:2IC7s3KVP6ottxQUnkm4eS',
       'spotify:album:0OcMap99vLEeGkBCfCwRwS',
       'spotify:album:7mCeLbChyegbRwwKK5shJs',
       'spotify:album:5MS3MvWHJ3lOZPLiMxzOU6',
       'spotify:album:2P2Xwvh2xWXIZ1OWY9S9o5',
       'spotify:album:1YwzJz7CrV9fd9Qeb6oo1d',
       'spotify:album:6HcU64bPPXTHIbWmGblIkT',
       'spotify:album:0C0Vs4XobImmqpr6kIasde',
       'spotify:album:6CvBb1XqN0igtQrWrbXD80',
       'spoti

In [19]:
final_df[final_df["album_name"].str.contains("Astro", case=False)]

,album_uri,album_name,total_tracks,release_date,artist_uris,artist_names
139,spotify:album:41GuZcammIkupMPKH2OJ6I,ASTROWORLD,17,2018-08-03,[spotify:artist:0Y5tJX1MQlPlqiwlOH1tJY],[Travis Scott]
140,spotify:album:5qeikHFUgeO8f8HXm8k4cA,ASTROWORLD,17,2018-08-02,[spotify:artist:0Y5tJX1MQlPlqiwlOH1tJY],[Travis Scott]


In [4]:
final_df[final_df.duplicated(["album_name", "total_tracks"], keep=False)].sort_values("release_date")


,album_uri,album_name,total_tracks,release_date,artist_uris,artist_names
299,spotify:album:1g3Ek21j6qDWt2CtravhrX,The Blueprint 3,15,2009-09-08,[spotify:artist:3nFkdlSjzX9mRTtwJOzDYB],[JAY-Z]
298,spotify:album:2CUT0104gySOIvqwtXeFsX,The Blueprint 3,15,2009-09-08,[spotify:artist:3nFkdlSjzX9mRTtwJOzDYB],[JAY-Z]
280,spotify:album:6klUp8sQyRXGuJhqZu4PG3,My Beautiful Dark Twisted Fantasy,13,2010-01-01,[spotify:artist:5K4W6rqBFWDnAN6FQUkS6x],[Kanye West]
278,spotify:album:20r762YmB5HeofjMCiPMLv,My Beautiful Dark Twisted Fantasy,13,2010-11-22,[spotify:artist:5K4W6rqBFWDnAN6FQUkS6x],[Kanye West]
271,spotify:album:2P2Xwvh2xWXIZ1OWY9S9o5,Watch The Throne (Deluxe),16,2011-08-08,"[spotify:artist:3nFkdlSjzX9mRTtwJOzDYB, spotif...","[JAY-Z, Kanye West]"
...,...,...,...,...,...,...
363,spotify:album:2vM2H6lyPfitmNZlc1Vrxv,Me vs. Myself (Deluxe),23,2022-12-12,[spotify:artist:31W5EY0aAly4Qieq6OFu6I],[A Boogie Wit da Hoodie]
344,spotify:album:4bJOIOKUAXFamjkGQwiVH6,Lil Durk Presents: Loyal Bros 2,23,2022-12-16,"[0, spotify:artist:3hcs9uc56yIGFCSy9leWe7]","[Only The Family, Lil Durk]"
343,spotify:album:4lXLULUDQvqV1yHtLW5fyL,Lil Durk Presents: Loyal Bros 2,23,2022-12-16,"[0, spotify:artist:3hcs9uc56yIGFCSy9leWe7]","[Only The Family, Lil Durk]"
99,spotify:album:43GclyRDmDm52SA342WoAm,I Am Music,18,2023-03-31,[spotify:artist:55Aa2cqylxrFIXC767Z865],[Lil Wayne]
